In [3]:
import torch
import torchvision
import cv2 
import numpy as np
import os

In [14]:
# Extracts the frames of a video
temp_video_file = os.path.join('labeled', "0.hevc")

def extract_frames(video_file):
    if not os.path.exists("data"):
        os.mkdir("data")
    cap = cv2.VideoCapture(video_file)   
    frames = []

    while(cap.isOpened  ()):
        frameId = cap.get(1) 
        ret, frame = cap.read()

        if (ret != True):
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray)

    np.save(os.path.join('data', os.path.split(video_file)[1].split('.')[0] + ".npy"), frames)
    
    cap.release()

# extract_frames(temp_video_file)

In [47]:
# frame_data = np.load(os.path.join("data", "0.npy"))
pitch_yaw_data = open(os.path.join("labeled", "0.txt"), 'r').read().split('\n')

In [50]:
print(pitch_yaw_data[0].split(' '))

['3.346066188150387949e-02', '3.149205029088487234e-02']
